In [1]:
!pip install PyPDF2
!pip install pdfminer.six

In [2]:
import PyPDF2
from pdfminer.high_level import extract_text

import pandas as pd
from copy import copy

In [3]:
def rus(text, alphabet=set('абвгдеёжзийклмнопрстуфхцчшщъыьэюя')):
    return not alphabet.isdisjoint(text.lower())

In [4]:
def eng(text, alphabet=set('abcdefghijklmnopqrstuvwxyz')):
    return not alphabet.isdisjoint(text.lower())

In [139]:
def find_extract(text: str, text_to_find: str, bias_=0, without_new_str=False, until=None):

    idx = text.find(text_to_find) + len(text_to_find) + bias_

    list_ = []

    if until is None:
        while not(eng(text[idx])):
            if text[idx] != '\n':
                list_.append(text[idx])
            else:
                if without_new_str:
                    break
            idx += 1
    else:
        while not(text[idx:].find(until) == 0):
            list_.append(text[idx])
            idx += 1

    return(''.join(list_[:-1]))

In [6]:
def parse_marks(text):

    idx = 0
    list_ = []

    for i in range(len(text)):
        if rus(text[i]) or text[i] == ' ':
            list_.append(text[i])

    list_ = ''.join(list_).split(' ')

    return [x for x in list_ if x in ('удовлетворительно', 'хорошо', 'отлично', 'зачтено')]

In [7]:
def parse_diploma_supplement(pdf_path):

    pdf_file = open(pdf_path, 'rb')
    text = extract_text(pdf_file, password='pass', page_numbers=None, maxpages=0,
                        caching=True, codec='utf-8', laparams=None)

    general_info = pd.DataFrame([], columns=['family_name', 'date_of_birth', 'access_requirements', 'admitted_to',
                                             'graduated_from', 'level_of_education', 'name_of_qualification',
                                             'main_field_of_study', 'academic_programme_title', 'professional_trajectory',
                                'official_length_of_programme', 'mode_of_study', 'languages_of_instruction_examinations',
                                             'graduation_project_theme', 'scientific_supervisor', 'graduation_project_mark', 'all_marks'])

    subjects_list = []
    subjects_marks_begin_idx = text[text.find('Конец перечня / End of the list'):].find('\x0c') + text.find('Конец перечня / End of the list') + 2
    subjects_begin_idx = subjects_marks_begin_idx

    for i in range(3):

        marks_begin_idx = min(text[subjects_begin_idx:].find('удовлетворительно'),
                              text[subjects_begin_idx:].find('хорошо'),
                              text[subjects_begin_idx:].find('отлично'),
                              text[subjects_begin_idx:].find('зачтено')) + subjects_begin_idx

        idx = marks_begin_idx
        count_n = 0
        while True:
            if text[idx] == '\n':
                count_n += 1
            idx -= 1
            if count_n >= 8:
                break
        subjects_end_idx = idx

        text1 = text[subjects_begin_idx: subjects_end_idx]

        for s in text1.split('\n\n'):
            subjects_list.append(s.split(' \n')[0])

        subjects_list1 = []
        for el in subjects_list:
            for symb in el:
                drop = True
                if rus(symb):
                    drop = False
                    break
            if not drop:
                subjects_list1.append(el)
        subjects_list = subjects_list1.copy()

        subjects_begin_idx = text[subjects_begin_idx:].find('\x0c') + subjects_begin_idx + 1

    marks_end_idx = text[subjects_marks_begin_idx:].find('Конец перечня') + subjects_marks_begin_idx
    marks_list = parse_marks(text[subjects_marks_begin_idx:marks_end_idx])

    marks_dict = dict()
    for i, key in enumerate(subjects_list):
        marks_dict[key] = marks_list[i]

    general_info.loc[len(general_info)] = [
        find_extract(text, 'Family name(s), given name(s) '),
        find_extract(text, 'Дата рождения (день/месяц/год)  '),
        find_extract(text, 'Access requirements '),
        find_extract(text, 'Admitted to '),
        find_extract(text, 'Graduated from '),
        find_extract(text, 'Level of education '),
        find_extract(text, 'Name of qualification '),
        find_extract(text, 'Main field of study '),
        find_extract(text, 'Academic programme title '),
        find_extract(text, 'Профессиональная траектория (профессиональные траектории): '),
        find_extract(text, 'Нормативный срок освоения ', without_new_str=True),
        find_extract(text, 'Форма обучения  ', without_new_str=True),
        find_extract(text, 'Language(s) of instruction / \nexaminations \n\n', without_new_str=True, until='/'),
        find_extract(text, 'Completion and defense \nof graduation project \n\n', until='\nНаучный руководитель').replace('\n', ''),
        find_extract(text, 'Научный руководитель: '),
        find_extract(text[text.find('ECTS \ngrade \n\nGrade') + len('ECTS \ngrade \n\nGrade'):], 'ECTS \ngrade \n\nGrade ')[4:],
        marks_dict
    ]

    return general_info

In [9]:
pdf_path = 'РыбаковСергейВячеславович_Supl-1.pdf'

parse_diploma_supplement(pdf_path)

,family_name,date_of_birth,access_requirements,admitted_to,graduated_from,level_of_education,name_of_qualification,main_field_of_study,academic_programme_title,professional_trajectory,official_length_of_programme,mode_of_study,languages_of_instruction_examinations,graduation_project_theme,scientific_supervisor,graduation_project_mark,all_marks
0,Рыбаков Сергей Вячеславович,05.07.2000,"Аттестат о среднем общем образовании, выданный...",Федеральное государственное бюджетное образова...,Федеральном государственном бюджетном образова...,бакалавриат,Бакалавр,01.03.02 Прикладная математика и информатика,"Прикладная математика, фундаментальная информа...",Математическое и программное обеспечение вычис...,4 года,очная,"English, Russian",Разработка масштабируемой архитектуры игрового...,кандидат технических наук И.С.Блеканов,отлично,"{'1С: Предприятие': 'отлично', 'Алгебра': 'удо..."


In [10]:
import unittest

In [192]:
class TestPDFParserFunctions(unittest.TestCase):

  def test_rus(self):
    self.assertTrue(rus("Привет"))
    self.assertFalse(rus("Hello"))
    self.assertFalse(rus("1234"))
    self.assertFalse(rus(""))

  def test_eng(self):
    self.assertTrue(eng("Hello"))
    self.assertFalse(eng("Привет"))
    self.assertFalse(eng("1234"))
    self.assertFalse(eng(""))

  def test_find_extract(self):
    text = 'This is english text. А это русский текст. Now you read another text. Другой \nрусский текст. End.'

    self.assertEqual(find_extract(text, 'text.', bias_=0, without_new_str=False, until=None),  ' А это русский текст.')
    self.assertEqual(find_extract(text, ' another text.', bias_=0, without_new_str=False, until=None), ' Другой русский текст.')
    self.assertEqual(find_extract(text, 'text.', bias_=0, without_new_str=False, until="т"), ' А ')
    self.assertEqual(find_extract(text, 'text.', bias_=1, without_new_str=False, until="т"), 'А ')
    self.assertEqual(find_extract(text, 'another text.', bias_=0, without_new_str=True, until=None), ' Другой')
    self.assertEqual(find_extract(text, 'another text.', bias_=1, without_new_str=True, until=None), 'Другой')
    self.assertEqual(find_extract(text, 'another text.', bias_=1, without_new_str=False, until=None), 'Другой русский текст.')
    self.assertEqual(find_extract(text, 'another text.', bias_=1, without_new_str=False, until='текст.'), 'Другой \nрусский')
    self.assertEqual(find_extract(text, 'another text.', bias_=1, without_new_str=True, until='й'), 'Друг')

  def test_parse_marks(self):
    text_with_marks = "Оценка по предмету 1: удовлетворительно \nОценка по предмету 2: отлично"
    self.assertEqual(parse_marks(text_with_marks), ["удовлетворительно", "отлично"])

    text_without_marks = "No marks are available."
    self.assertEqual(parse_marks(text_without_marks), [])

    text_with_symbols = "Оценки: предмет 1: удовлетворительно; предмет 2:хорошо, предмет3: отлично."
    self.assertEqual(parse_marks(text_with_symbols), ['удовлетворительно', 'хорошо', 'отлично'])

    text_with_new_str = "Оценки:\nПредмет 1: удовлетворительно\nПредмет 2:хорошо\nПредмет 3: отлично\n"
    self.assertEqual(parse_marks(text_with_new_str), ['отлично'])


  def test_valid_pdf(self):
    pdf_path = 'РыбаковСергейВячеславович_Supl-1.pdf'
    result = parse_diploma_supplement(pdf_path)

    self.assertIsInstance(result, pd.DataFrame)
    self.assertFalse(result.empty)

  def test_invalid_pdf_path(self):
    pdf_path = 'path/to/nonexistent.pdf'

    with self.assertRaises(FileNotFoundError):
      parse_diploma_supplement(pdf_path)


In [193]:
unittest.main(argv=[''], verbosity=2, exit=False)

test_eng (__main__.TestPDFParserFunctions) ... ok
test_find_extract (__main__.TestPDFParserFunctions) ... ok
test_invalid_pdf_path (__main__.TestPDFParserFunctions) ... ok
test_parse_marks (__main__.TestPDFParserFunctions) ... ok
test_rus (__main__.TestPDFParserFunctions) ... ok
test_valid_pdf (__main__.TestPDFParserFunctions) ... ok
test_invalid_pdf_path (__main__.TestParseDiplomaSupplement) ... ok
test_valid_pdf (__main__.TestParseDiplomaSupplement) ... ok

----------------------------------------------------------------------
Ran 8 tests in 4.366s

OK
